In [1]:
!pip install torch-fidelity
!pip install torchmetrics[image]

  Using cached torch-2.3.0-cp39-cp39-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.

In [2]:
generate_images_folder = './e2e_64/96/images'
orig_images_folder = './e2e_64/vae'

gen_image_prefix = 'output_'
orig_images_prefix = 'output_'

In [3]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torchmetrics.image.fid import FrechetInceptionDistance


# 이미지 불러오기 및 텐서로 변환하는 함수
def load_images_from_folder(folder, start_dir):
    images = []
    transform = transforms.Compose([
        transforms.Resize((64, 64)),  # 이미지 크기를 128x128로 조정
        transforms.ToTensor(),
        transforms.Lambda(lambda x: (x * 255).byte())
    ])
    print(folder)
    for filename in os.listdir(folder):
        print(filename)
        if filename.startswith(start_dir):
            img_path = os.path.join(folder, filename)
            
            if not os.path.isfile(img_path):
                    continue
                    
            with Image.open(img_path) as img:                    
                img = transform(img)
                images.append(img) 
    return torch.stack(images)

# vae_로 시작하는 이미지 불러오기
vae_images = load_images_from_folder(generate_images_folder, gen_image_prefix)

# orig_로 시작하는 이미지 불러오기
orig_images = load_images_from_folder(orig_images_folder, orig_images_prefix)

# FID 인스턴스 생성
fid = FrechetInceptionDistance(feature=64)

# 실제(real) 이미지로 orig_ 이미지 업데이트
fid.update(orig_images, real=True)

# 생성된(generated) 이미지로 vae_ 이미지 업데이트
fid.update(vae_images, real=False)

# FID 계산
fid_value = fid.compute()


./e2e_64/96/images
output_gpu_0_0_0_3.png
output_gpu_0_0_6_3.png
output_gpu_0_0_2_3.png
output_gpu_0_0_24_1.png
output_gpu_0_0_15_3.png
output_gpu_0_0_5_1.png
output_gpu_0_0_12_3.png
output_gpu_0_0_7_3.png
output_gpu_0_0_16_2.png
output_gpu_0_0_15_1.png
output_gpu_0_0_18_1.png
output_gpu_0_0_9_3.png
output_gpu_0_0_20_3.png
output_gpu_0_0_8_3.png
output_gpu_0_0_4_0.png
output_gpu_0_0_14_1.png
output_gpu_0_0_15_2.png
output_gpu_0_0_0_1.png
output_gpu_0_0_24_2.png
output_gpu_0_0_17_2.png
output_gpu_0_0_21_2.png
output_gpu_0_0_18_2.png
output_gpu_0_0_17_3.png
output_gpu_0_0_11_3.png
output_gpu_0_0_8_2.png
output_gpu_0_0_10_1.png
output_gpu_0_0_2_0.png
output_gpu_0_0_13_1.png
output_gpu_0_0_9_2.png
output_gpu_0_0_14_3.png
output_gpu_0_0_18_3.png
output_gpu_0_0_1_3.png
output_gpu_0_0_6_1.png
output_gpu_0_0_18_0.png
output_gpu_0_0_14_0.png
output_gpu_0_0_1_0.png
output_gpu_0_0_4_1.png
output_gpu_0_0_11_1.png
output_gpu_0_0_23_3.png
output_gpu_0_0_1_2.png
output_gpu_0_0_19_3.png
output_gpu_0_0

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 331MB/s]


In [4]:
fid_value

tensor(7.7262)

In [5]:
def calculate_inception_score(images, batch_size=32, splits=10):
       # Inception 모델 로드
    inception_model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT, transform_input=False)
    inception_model.eval()

    # 이미지 변환 정의
    preprocess = transforms.Compose([
        transforms.ToPILImage(),        # 텐서를 PIL 이미지로 변환
        transforms.Resize((299, 299)),  # 이미지 크기 조정
        transforms.ToTensor(),          # 텐서로 변환
    ])

    # 이미지 텐서를 batch_size로 나누기
    n_images = images.size(0)
    preds = []
    for i in range(0, n_images, batch_size):
        batch = images[i:i + batch_size]
        batch = torch.stack([preprocess(img) for img in batch])  # 각 이미지를 전처리
        batch = batch.float() / 255.0  # 0~1 범위로 정규화
        with torch.no_grad():
            pred = inception_model(batch)
            preds.append(F.softmax(pred, dim=1).cpu())

    preds = torch.cat(preds, dim=0)

    # Inception Score 계산
    scores = []
    for i in range(splits):
        part = preds[i * (n_images // splits): (i + 1) * (n_images // splits), :]
        py = torch.mean(part, dim=0)
        scores.append(torch.exp(torch.mean(torch.sum(part * torch.log(part / py), dim=1))))

    return torch.mean(torch.tensor(scores)), torch.std(torch.tensor(scores))

In [6]:
import os
from PIL import Image
import torch
from torchvision import transforms, models
from torch.nn import functional as F

generated_images = load_images_from_folder(generate_images_folder, gen_image_prefix)

# Inception Score 계산
mean_score, std_score = calculate_inception_score(generated_images)

print(f"Inception Score: {mean_score.item()} ± {std_score.item()}")

./e2e_64/96/images
output_gpu_0_0_0_3.png
output_gpu_0_0_6_3.png
output_gpu_0_0_2_3.png
output_gpu_0_0_24_1.png
output_gpu_0_0_15_3.png
output_gpu_0_0_5_1.png
output_gpu_0_0_12_3.png
output_gpu_0_0_7_3.png
output_gpu_0_0_16_2.png
output_gpu_0_0_15_1.png
output_gpu_0_0_18_1.png
output_gpu_0_0_9_3.png
output_gpu_0_0_20_3.png
output_gpu_0_0_8_3.png
output_gpu_0_0_4_0.png
output_gpu_0_0_14_1.png
output_gpu_0_0_15_2.png
output_gpu_0_0_0_1.png
output_gpu_0_0_24_2.png
output_gpu_0_0_17_2.png
output_gpu_0_0_21_2.png
output_gpu_0_0_18_2.png
output_gpu_0_0_17_3.png
output_gpu_0_0_11_3.png
output_gpu_0_0_8_2.png
output_gpu_0_0_10_1.png
output_gpu_0_0_2_0.png
output_gpu_0_0_13_1.png
output_gpu_0_0_9_2.png
output_gpu_0_0_14_3.png
output_gpu_0_0_18_3.png
output_gpu_0_0_1_3.png
output_gpu_0_0_6_1.png
output_gpu_0_0_18_0.png
output_gpu_0_0_14_0.png
output_gpu_0_0_1_0.png
output_gpu_0_0_4_1.png
output_gpu_0_0_11_1.png
output_gpu_0_0_23_3.png
output_gpu_0_0_1_2.png
output_gpu_0_0_19_3.png
output_gpu_0_0

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:05<00:00, 19.3MB/s] 


Inception Score: 1.0000483989715576 ± 1.28197261801688e-05
